In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils import spectral_norm
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
import pandas as pd

In [2]:
# Load data
num_seeds = 8
data_dir = os.getenv('TRUNK_DATA', '/home/trunk/Documents/trunk-stack/stack/main/data')

us_df = pd.read_csv(os.path.join(data_dir, 'trajectories/steady_state/control_inputs_beta_seed0.csv'))
for seed in range(1, num_seeds + 1):
    us_df = pd.concat([us_df, pd.read_csv(os.path.join(data_dir, f'trajectories/steady_state/control_inputs_beta_seed{seed}.csv'))])
us_df = us_df.drop(columns=['ID'])

# Observations
ys_df = pd.read_csv(os.path.join(data_dir, 'trajectories/steady_state/observations_steady_state_beta_seed0.csv'))
for seed in range(1, num_seeds + 1):
    ys_df = pd.concat([ys_df, pd.read_csv(os.path.join(data_dir, f'trajectories/steady_state/observations_steady_state_beta_seed{seed}.csv'))])
ys_df = ys_df.drop(columns=['ID'])

rest_positions = np.array([0.1005, -0.10698, 0.10445, -0.10302, -0.20407, 0.10933, 0.10581, -0.32308, 0.10566])
ys_df = ys_df - rest_positions
N = len(ys_df)
if len(us_df) != N:
    us_df = us_df[:N]

# Convert to numpy arrays
us = us_df.to_numpy()
ys = ys_df.to_numpy()

G = np.linalg.lstsq(ys, us, rcond=None)[0].T
np.save(os.path.join(data_dir, 'models/ik/y2u_8seeds.npy'), G)

In [2]:
class MLP(nn.Module):
    def __init__(self,
                 num_inputs: int = 9,
                 num_outputs: int = 6,
                 num_neurons: list = [32, 32],
                 act: nn.Module = nn.Tanh(),  # should be relu for spectral norm to make sense
                 spectral_normalize: bool = False):
        super(MLP, self).__init__()

        layers = []

        # Input layer
        if spectral_normalize:
            input_layer = spectral_norm(nn.Linear(num_inputs, num_neurons[0]))
        else:
            input_layer = nn.Linear(num_inputs, num_neurons[0])
        layers.append(input_layer)
        layers.append(act)
        
        # Hidden layers
        for i in range(len(num_neurons) - 1):
            if spectral_normalize:
                hidden_layer = spectral_norm(nn.Linear(num_neurons[i], num_neurons[i + 1]))
            else:
                hidden_layer = nn.Linear(num_neurons[i], num_neurons[i + 1])
            layers.append(hidden_layer)
            layers.append(act)
        
        # Output layer
        if spectral_normalize:
            output_layer = spectral_norm(nn.Linear(num_neurons[-1], num_outputs))
        else:
            output_layer = nn.Linear(num_neurons[-1], num_outputs)
        
        layers.append(output_layer)
        self.layers = nn.ModuleList(layers)

    def forward(self, y):
        x = y
        for layer in self.layers:
            x = layer(x)
        u = x
        return u

In [3]:
data_dir = os.getenv('TRUNK_DATA', '/home/trunk/Documents/trunk-stack/stack/main/data')

neural_ik_model = MLP()
neural_ik_model.load_state_dict(torch.load(os.path.join(data_dir, 'models/ik/neural_ik_model_state.pth'), weights_only=False))
neural_ik_model.eval()

MLP(
  (layers): ModuleList(
    (0): Linear(in_features=9, out_features=32, bias=True)
    (1): Tanh()
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): Tanh()
    (4): Linear(in_features=32, out_features=6, bias=True)
  )
)

In [7]:
zf_sample = [0.0003403276205062866, -0.10695958137512207,-0.0010286569595336914, 0.0024021118879318237, -0.20283722877502441,-0.019539475440979004, -0.007723316550254822, -0.2861044406890869, -0.10018789768218994]
avp_offset = torch.tensor([0, -0.10698, 0, 0, -0.20407, 0, 0, -0.32308, 0], dtype=torch.float)

with torch.no_grad():
    # Forward pass
    nn_output = neural_ik_model(torch.tensor(zf_sample, dtype=torch.float)-avp_offset)
u_opt = nn_output.numpy()
print(u_opt)

[-1.7440327   0.3288924  -0.20543166 -1.6499428  -2.9646416   1.1011922 ]


In [8]:
torch.tensor(zf_sample, dtype=torch.float)-avp_offset

tensor([ 3.4033e-04,  2.0422e-05, -1.0287e-03,  2.4021e-03,  1.2328e-03,
        -1.9539e-02, -7.7233e-03,  3.6976e-02, -1.0019e-01])

In [9]:
u1, u2, u3, u4, u5, u6 = u_opt[0], u_opt[1], u_opt[2], u_opt[3], u_opt[4], u_opt[5]

u1_vec = u1 * np.array([-np.cos(15 * np.pi/180), np.sin(15 * np.pi/180)])
u2_vec = u2 * np.array([np.cos(45 * np.pi/180), np.sin(45 * np.pi/180)])
u3_vec = u3 * np.array([-np.cos(15 * np.pi/180), -np.sin(15 * np.pi/180)])
u4_vec = u4 * np.array([-np.cos(75 * np.pi/180), np.sin(75 * np.pi/180)])
u5_vec = u5 * np.array([np.cos(45 * np.pi/180), -np.sin(45 * np.pi/180)])
u6_vec = u6 * np.array([-np.cos(75 * np.pi/180), -np.sin(75 * np.pi/180)])


vector_sum = (
    0.75 * (u3_vec + u4_vec) +
    1.0 * (u2_vec + u5_vec) +
    1.4 * (u1_vec + u6_vec)
)
norm_value = np.linalg.norm(vector_sum)

In [10]:
norm_value

1.1031566923545795